In [2]:
import numpy as np
np.linspace(2.85,3.45,7)

array([2.85, 2.95, 3.05, 3.15, 3.25, 3.35, 3.45])

In [ ]:
import pandas as pd

# 1. 读取 CSV 文件
df = pd.read_csv("input.csv")

# 2. 添加新列（示例：静态值或动态计算）
# --------------------------------
# 示例 1：添加静态值（所有行的新列值为固定值）
df["新列名称"] = "固定值"

# 示例 2：基于现有列计算新列（例如两列相加）
df["总和"] = df["列A"] + df["列B"]

# 示例 3：按条件生成新列（例如分类标签）
df["分类"] = df["分数"].apply(lambda x: "及格" if x >= 60 else "不及格")

# 3. 保存到新文件（或覆盖原文件）
df.to_csv("output.csv", index=False, encoding="utf-8")